# Sismique active S : Conversion de formats

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd

from scipy import stats

import shutil

## Lecture de la géométrie du profil

In [2]:
from geometry import *

In [3]:
!cat geometry.py

import numpy as np
import pandas as pd
import os

gnss = pd.read_csv('data/gnss_sismique_s.csv')
gnss['geophone'] = gnss['Name'].map(lambda e: int(e.split('.')[-1]))
gnss = gnss.set_index('geophone')
gnss = gnss.sort_index()

files_pointage = [f for f in os.listdir('data/pointage') if f.endswith('txt')]

dx = 4
offset = 2
ng = len(gnss['A'].values)
ns = len(files_pointage)
x_geophs = np.arange(ng)*dx # géophones espacés de 4m
x_shots = np.arange(ns)*dx + offset # sources espacées de 4m entre 2 géophones
x = np.concatenate((x_geophs, x_shots))
x.sort()
y_geophs = gnss['A'].values
y = np.interp(x, x_geophs, y_geophs) # interpolation de l'altitude des capteurs (mesurées) pour estimer celle des sources
y_shots = y[0:-1:2]

In [4]:
gnss.head()

,Name,Latitude,Longitude,EllipsoidHeight,TimeStamp,X,Y,A
geophone,,,,,,,,
1,ONDES.GR1.1,48.491419,7.288737,713.242974,2024-09-04T15:10:04.420,6.829844e+06,1.016730e+06,664.504873
2,ONDES.GR1.2,48.491391,7.288704,713.303219,2024-09-04T15:10:28.280,6.829841e+06,1.016728e+06,664.564993
3,ONDES.GR1.3,48.491363,7.288673,713.271782,2024-09-04T15:10:46.900,6.829837e+06,1.016726e+06,664.533426
4,ONDES.GR1.4,48.491334,7.288640,713.350197,2024-09-04T15:11:05.850,6.829834e+06,1.016723e+06,664.611712
5,ONDES.GR1.5,48.491305,7.288607,713.588551,2024-09-04T15:11:25.210,6.829831e+06,1.016721e+06,664.849936


In [5]:
print('Nombre de geophones', ng)
print('Nombre de shots (sources)', ns)
print('Nombre total de données', ns*ng)

Nombre de geophones 21
Nombre de shots (sources) 20
Nombre total de données 420


In [6]:
path_pointage = 'data/pointage'

## Pygimli
data.sgt

In [7]:
path_pygimli = 'data/sgt'

In [8]:
xy = np.array((x, y)).T # positions de tous les capteurs et receveurs

In [9]:
s = []
g = []
t = []
dmin = 0 # distance min from source (all bellow removed)
fmt = "{:03d}000.txt"
files_pointage = os.listdir(path_pointage)
for i, x_shot in enumerate(x_shots):
    filename = fmt.format(x_shot)
    file = f'{path_pointage}/{filename}'
    if filename not in files_pointage:
        print(f'File {file} not found skipping.')
        continue
    ts = np.loadtxt(file, usecols=0, dtype=float)
    ks = np.loadtxt(file, usecols=1, dtype=int)
    for j, k in enumerate(ks):
        x_geoph = x_geophs[k-1]
        if x_shot - dmin <= x_geoph <= x_shot + dmin:
            continue
        t.append(ts[j])
        s.append(2*(i+1))
        g.append(2*k-1)
s = np.array(s)
g = np.array(g)
t = np.array(t)

In [10]:
print('Nombre de geophones', ng)
print('Nombre de shots (sources)', ns)
print('Nombre total de positions', len(xy)) # car les shots sont aux positions des capteurs donc on ne les compte pas
print('Nombre total de données', ns*ng)
print('Nombre total de données réelles', len(t)) # en réalité on en retire 1 par shot soit 20

Nombre de geophones 21
Nombre de shots (sources) 20
Nombre total de positions 41
Nombre total de données 420
Nombre total de données réelles 420


In [11]:
np.savetxt(f'{path_pygimli}/positions.txt', xy)
np.savetxt(f'{path_pygimli}/shots.txt', s, fmt='%d')
np.savetxt(f'{path_pygimli}/geophones.txt', g, fmt='%d')
np.savetxt(f'{path_pygimli}/times.txt', t)
np.savetxt(f'{path_pygimli}/sgt.txt', np.array([s, g, t]).T) # , delimiter='\t', fmt='%.5f'

In [12]:
with open(f'{path_pygimli}/data.sgt', 'w') as f:
    f.write(f'{len(xy)} # shot/geophone points\n')
    f.write('#x	y\n')
    #f.write(open('xy', 'r').read())
    for pos in xy:
        x, y = pos
        f.write(f'{x} {y}\n')
    f.write(f'{len(t)} # measurements\n')
    f.write('#s	g	t\n')
    #f.write(open('sgt', 'r').read())
    for i in range(len(t)):
        f.write(f'{s[i]} {g[i]} {t[i]}\n')

## Rayinvr

[Rayinvr](https://github.com/hzhu212/rayinvr)

In [13]:
path_rayinvr = 'data/rayinvrmodel'

In [14]:
slopes_ranges = np.loadtxt('data/modele_vitesses/slopes_ranges.txt')
v0, v1, v2, v3 = np.loadtxt('data/modele_vitesses/vitesses.txt').T*1e3
z1, z2, z3 = np.loadtxt('data/modele_vitesses/epaisseurs.txt').T

In [15]:
def find_layer(x_geoph, geophones):
    """
    find which is the corresponding layer of the measurement at this point
    """
    for l in range(1, len(geophones)//2):
        x_geoph1, x_geoph2 = geophones[2*l-2:2*l]
        if x_geoph1 == x_geoph2:
            pass
        if x_geoph1 <= x_geoph <= x_geoph2:
            return max(1, (l-1)%(len(geophones)//4))
    return 1

In [16]:
# tx : travel times
txin = []
uncertainty = 1.5 # on the picks

for i, x_shot in enumerate(x_shots):
    shot_data = []
    file = f"data/pointage/{x_shot:03d}000.txt"
    ts = np.loadtxt(file, usecols=0, dtype=float)*1000
    ks = np.loadtxt(file, usecols=1, dtype=int)
    for j, k in enumerate(ks):
        x_geoph = x_geophs[k-1]
        geophones = slopes_ranges[i]
        layer = find_layer(x_geoph, geophones)
        shot_data.append([x_geoph, ts[j], uncertainty, layer])
    if shot_data[0][0] < x_shot:
        shot_data.insert(0, [x_shot, -1, 0, 0])
    shot_data.insert(i+2, [x_shot, 1, 0, 0])
    txin += shot_data
txin.append([0, 0, 0, -1])

np.savetxt(f'{path_rayinvr}/tx.ori', txin, fmt="% 10.3f% 10.3f% 10.3f% 10d")

In [17]:
# fix spacing on tx.ori (fortran is very strict on the spacing..)
txin = np.loadtxt(f'{path_rayinvr}/tx.ori')
np.savetxt(f'{path_rayinvr}/tx.ori', txin, fmt="% 10.3f% 10.3f% 10.3f% 10d")

In [18]:
def create_layer_row(k, x_layer, y_layer=0, vary=False, vori='', ncolmax=10):
    """
    create one line of the v.ori velocity model for rayinvr
    k : int, number of the layer starting at 1
    x_layer : array_like, horizontal position for the mesh
    y_layer : array_like or float_like, value of the parameter at the corresponding point (depth or velocity)
    vary : bool, should the parameter evolves thru iterations or is it fixed
    vori : str, string to append data on
    ncolmax : number of columns to write on, default 10
    """
    y_layer = np.zeros_like(x_layer) + y_layer
    v_layer = np.zeros_like(x_layer, dtype=int) + vary
    n_cols = len(x_layer)//10+1
    fmt_float = '{: 8.3f}'
    for i in range(n_cols):
        vori += f'{k: 2d} '+''.join(fmt_float.format(e) for e in x_layer[i*10:(i+1)*10])+'\n'
        vori += f'{int(i!=n_cols-1): 2d} '+''.join(fmt_float.format(e) for e in y_layer[i*10:(i+1)*10])+'\n'
        vori += 3*' '+''.join(f'{int(e): 7d}' for e in v_layer[i*10:(i+1)*10])+'\n'
    return vori

In [19]:
def create_layer(k, x_layer, depth_layer, velocity_layer_top, velocity_layer_bottom, varies=(False, False, False), vori='', ncolmax=10):
    """
    create one layer of the v.ori velocity model for rayinvr
    k : int, number of the layer starting at 1
    x_layer : array_like, horizontal position for the mesh
    depth_layer : array_like or float_like, depth of the layer at the corresponding point
    velocity_layer_top : array_like or float_like, velocity of the top of the layer at the corresponding point
    velocity_layer_bottom : array_like or float_like, velocity of the top of the layer at the corresponding point
    varies : (bool, bool, bool), should the depth, top velocity or bottom velocity evolves thru iterations or is it fixed
    vori : str, string to append data on
    ncolmax : number of columns to write on, default 10
    """
    vori = create_layer_row(x_layer=x_layer, y_layer=depth_layer, k=k, vary=varies[0], vori=vori) # Layer depth
    vori = create_layer_row(x_layer=x_layer, y_layer=velocity_layer_top, k=k, vary=varies[1], vori=vori) # Layer velocity top
    vori = create_layer_row(x_layer=x_layer, y_layer=velocity_layer_bottom, k=k, vary=varies[2], vori=vori) # Layer velocity bottom
    return vori

In [20]:
# v : velocity model
k = 1 # First layer
x_borders = 5
x_layer = np.concatenate([[x_geophs[0] - x_borders], x_geophs, [x_geophs[-1] + x_borders]])
depth_layer = np.max(y_geophs) - np.interp(x_layer, x_geophs, y_geophs)
velocity_layer_top = np.interp(x_layer, x_shots, v0)
velocity_layer_bottom = np.interp(x_layer, x_shots, v1) - .05
varies = (False, False, False)
vori = create_layer(k=k, x_layer=x_layer, depth_layer=depth_layer, velocity_layer_top=velocity_layer_top, velocity_layer_bottom=velocity_layer_bottom, varies=varies)

k = 2 # Second layer
depth_layer = depth_layer + np.interp(x_layer, x_shots, z2)
velocity_layer_top = np.interp(x_layer, x_shots, v2)
velocity_layer_bottom = velocity_layer_top + .1
varies = (True, False, False)
vori = create_layer(k=k, x_layer=x_layer, depth_layer=depth_layer, velocity_layer_top=velocity_layer_top, velocity_layer_bottom=velocity_layer_bottom, varies=varies, vori=vori)

k = 3 # Third layer
depth_layer = depth_layer + 20
velocity_layer_top = 3.2
velocity_layer_bottom = 4
varies = (False, False, False)
vori = create_layer(k=k, x_layer=x_layer, depth_layer=depth_layer, velocity_layer_top=velocity_layer_top, velocity_layer_bottom=velocity_layer_bottom, varies=varies, vori=vori)

vori += ' 4   -5.00  85.00\n'
vori += ' 0   40.00  40.00\n'
with open(f'{path_rayinvr}/v.ori', 'w') as f:
    f.write(vori)

In [21]:
# r : rays parameters
print(f"ishot={'2,'*ns}")
print(f"xshot={','.join(map(str, map(float, x_shots)))},")

ishot=2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,
xshot=2.0,6.0,10.0,14.0,18.0,22.0,26.0,30.0,34.0,38.0,42.0,46.0,50.0,54.0,58.0,62.0,66.0,70.0,74.0,78.0,


In [22]:
# create input files from ori
filenames = ('tx', 'v', 'd', 'r')
for filename in filenames:
    shutil.copy2(f'{path_rayinvr}/{filename}.ori', f'{path_rayinvr}/{filename}.in')